<a href="https://colab.research.google.com/github/ratul41907/Food-and-Nutrition-Based-RAG-Chatbot/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-learn sentence-transformers transformers faiss-cpu pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 46.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive


drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
# List files in your Google Drive folder
os.listdir('/content/drive/MyDrive/')


['Colab Notebooks',
 'hsccc.pdf',
 'WEB_BASED_RESULT_PUBLICATION_SYSTEM_FOR_EDUCATION_BOARDS.pdf',
 'siuuu.jpg',
 'de.jpg',
 'Untitled presentation (1).gslides',
 'Untitled document (19).gdoc',
 'Copy of Year at a Glance - Template.gsheet',
 'Untitled document (18).gdoc',
 'plan.docx',
 'Lecture 4.pptx',
 'MS1 MAT 130 Summer 20023.pdf',
 'CGPA calculator.xlsx',
 'Weekly schedule planner1 (1).xlsx',
 'Weekly schedule planner1.xlsx',
 'Government.docx',
 'Assignment SFR1.gdoc',
 'Untitled document (17).gdoc',
 'Procedures.gdoc',
 'images.docx',
 'cover page-1 (1).docx',
 'Government.gdoc',
 'Sem 1.gdoc',
 'CW_23april.gdoc',
 'WalidurRahman_2311712642_HW13.gdoc',
 'Untitled document (16).gdoc',
 'Untitled document (15).gdoc',
 'Untitled document (14).gdoc',
 'Recording 2024-04-28 102549 (1).mp4',
 'MAY DAY.gsheet',
 'Untitled document (13).gdoc',
 'Untitled document (12).gdoc',
 'To-do list (1).gsheet',
 'Untitled spreadsheet (4).gsheet',
 'Copy of IC-Student-Schedule-Template_Google-Shee

In [ ]:
import os
# List files in your Google Drive folder
os.listdir('/content/drive/MyDrive/Colab Notebooks')


['Chatbot.ipynb',
 'Note1.ipynb',
 'Project.ipynb',
 'Untitled0.py',
 'ML.ipynb',
 'Untitled0.ipynb',
 'MLBasics.ipynb',
 'MLPart2.ipynb',
 'DataAdding Misisng.ipynb',
 'Data Standardization.ipynb',
 'Label Encoding.ipynb',
 'Handling Imbalanced Dataset.ipynb',
 'Frequencies.ipynb',
 'Untitled1.ipynb',
 'Rock vs Mine using ML.ipynb',
 'Diabetes Prediction.ipynb',
 'OSTAD,_A_1.ipynb',
 'OSTAD_A_2.ipynb',
 'Untitled2.ipynb',
 'beauty.parquet',
 'food.parquet']

In [ ]:
#library fot .parquet
!pip install pyarrow


In [ ]:
import dask.dataframe as dd

# Load the dataset using Dask (works for larger files)
file_path_food = '/content/drive/MyDrive/Colab Notebooks/food.parquet'
dask_df = dd.read_parquet(file_path_food)

# Take the first 50,000 rows
sampled_dask_df = dask_df.head(50000)  # Load the first 50,000 rows

# Display the first few rows to ensure the data loaded correctly
print(sampled_dask_df.head())

# Save the subset to a new Parquet file
sampled_dask_df.to_parquet('/content/drive/MyDrive/Colab Notebooks/food_sample.parquet')
print("Data saved to food_sample.parquet")


/usr/local/lib/python3.12/dist-packages/dask/dataframe/core.py:382: UserWarning: Insufficient elements for `head`. 50000 elements requested, only 35840 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(


   additives_n additives_tags allergens_tags   brands_tags   brands  \
0          NaN           None      [en:nuts]  [xx:bovetti]  Bovetti   
1          0.0             []             []      [lagg-s]   Lagg's   
2          0.0             []             []      [lagg-s]   Lagg's   
3          0.0             []             []   [xx:lagg-s]   Lagg's   
4          0.0             []             []      [lagg-s]   Lagg's   

                                          categories  \
0  Petit-déjeuners,Produits à tartiner,Produits à...   
1                                               null   
2  Plant-based foods and beverages, Beverages, Ho...   
3  Beverages and beverages preparations, Plant-ba...   
4                                               <NA>   

                                     categories_tags  \
0  [en:breakfasts, en:spreads, en:sweet-spreads, ...   
1                                          [en:null]   
2  [en:plant-based-foods-and-beverages, en:bevera...   
3  [en:bever